In [2]:
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import locale
import pandas as pd
import threading
import random
import urllib.parse

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}

In [6]:
df = pd.read_csv('../dataset.csv')
# scraped = pd.read_csv('./product.csv')
df = df.rename(columns={'shopid': 'ShopId', 'item_name': 'ProductName', 'price': 'Price',  'stock': 'Stock','item_description': 'Description', 'category': 'Category'})[['ShopId', 'ProductName', 'Price', 'Stock', 'Description', 'Category']]
df = df.drop_duplicates(subset=['ProductName']).reset_index(drop=True)

In [7]:

MAX_LINKS_TO_FETCH = 3

def get_image_urls(product):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)

    link = f'https://www.google.com/search?q={product}&hl=en-US&source=lnms&tbm=isch&sa=X&ved=2ahUKEwj7ldOXuuP9AhUlTmwGHdZhAeIQ_AUoAnoECAEQBA&biw=1277&bih=1363&dpr=1'
    driver.get(link)

    image_urls = set()
    # Scroll to the bottom of the page to load more images
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.implicitly_wait(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    thumbnails = driver.find_elements(By.CSS_SELECTOR, 'img.Q4LuWd')

    for img in thumbnails[0:MAX_LINKS_TO_FETCH]:
        try:
            img.click()
            driver.implicitly_wait(2)
        except Exception: 
            continue

        actual_images = driver.find_elements(By.CSS_SELECTOR,'img.n3VNCb')
        for actual_image in actual_images:
            src = actual_image.get_attribute('src')
            if src and 'http' in src and src not in image_urls:
                image_urls.append(src)

        if len(image_urls) >= MAX_LINKS_TO_FETCH:
            break
        
    driver.quit()
    return ','.join(image_urls)

In [9]:
prefix = 'https://www.google.com/search?q='
suffix = '&hl=en-US&source=lnms&tbm=isch&sa=X&ved=2ahUKEwj7ldOXuuP9AhUlTmwGHdZhAeIQ_AUoAnoECAEQBA&biw=1277&bih=1363&dpr=1'
products = df.ProductName.apply(str).apply(urllib.parse.quote_plus).values
# image_urls = df.image_urls.values

# i = 0
# thread_list = list()

# for product in products:
#     if pd.isna(scraped.image_urls.loc[i]):
#         link = f'{prefix}{product}{suffix}'
#         t = threading.Thread(target=get_image_urls, args=(i, link), daemon=True)
#         t.start()
#         time.sleep(max(random.random(), 0.1) + 0.2)
#         thread_list.append(t)
#     i += 1

# for thread in thread_list:
#     thread.join()
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for i, product in enumerate(products):
            # if pd.isna(image_urls[i]) or not image_urls[i]:
            if i <= 10000:
                futures.append(executor.submit(get_image_urls, product))
                print(f'\rStarted scraping on product #{i}', end='')
            else:
                break

        for i, future in enumerate(futures):
            image_urls = future.result()
            if image_urls:
                df.loc[i, 'image_urls'] = image_urls

            print(f'\rFinished scraping on product #{i}', end='')
            

print('Test completed!')

Started scraping on product #10000

In [4]:
scraped.image_urls.count()

10443

In [12]:
scraped.image_urls.count()

10680

In [ ]:
for product in products:
    if not product.image_urls:
        print(product)

In [25]:
if not scraped.image_urls.loc[118342]:
    print(True)

In [3]:
scraped

,ShopId,ProductName,Price,Stock,Description,Category,image_urls
0,210000,Golden mobile numbers,400.00,1,Unregistered prepaid cards1) 93355333 selling...,Mobile & Gadgets,NaN
1,210000,Golden Mobile Numbers,8.00,1,Unregistered prepaid cards. Can port to post p...,Mobile & Gadgets,https://media.karousell.com/media/photos/produ...
2,760000,101% AUTHENTIC BASEBALL CAPS,35.00,300,"PREORDER Takes about 23 weeks to arrive, will ...",Women's Apparel,https://media.karousell.com/media/photos/produ...
3,11272000,Hada Labo Super Hydrating Perfect Gel Moisturi...,28.72,8,Hada Labo Super Hydrating Perfect Gel Moisturi...,Health & Beauty,https://gphb01pdazurefileshare.blob.core.windo...
4,49082000,Women Lace Bra Bustier Crop Tops Soft Mesh Tri...,9.98,400,Material: Terylene Colors: BalckStyle: Sexy Br...,Women's Apparel,NaN
...,...,...,...,...,...,...,...
118342,13744999,Cute Flower Sleeveless Mesh Baby Girls Dress,7.57,0,Style: Cute Material: Cotton Fabric Type: Broa...,"Toys, Kids & Babies",NaN
118343,13744999,Babies Beanie Hat Han Edition Cotton Letter Pr...,1.40,0,Gender: Unisex Item Type: Baby Hat Material: C...,"Toys, Kids & Babies",NaN
118344,13744999,Women Back Brest Dress Package Hips Dress,7.70,0,Type: Dress Gender: Womens Size Type: Regular ...,Women's Apparel,NaN
118345,13744999,Heart Alloy Pendant Necklace for women,0.60,0,Gender: For Women Item Type: Pendant Necklaces...,Jewellery & Accessories,NaN


In [30]:
scraped.image_urls.loc[0]

''